In [10]:
try:
    import PIL.Image as Image
    import pathlib
    import tensorflow as tf
except Exception:
    pass
import tensorflow as tf
import numpy as np
import pandas as pd
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import cv2


tf.keras.utils.get_file(
    'Valid_spectral.zip', 'https://github.com/caiyuanhao1998/MST-plus-plus', extract=True)


image_root = pathlib.Path('/Users/dajineyland/.keras/datasets/Valid_spectral')
all_images_paths = list(image_root.glob('*'))
# print(all_images_paths)

# RGB_img = all_images_paths[c, -1]
# plt.figure(figsize=(5, 5))  # 전체 팝업창 크기
# for c in range(9):
#     plt.subplot(3, 3, c+1)  # 가로3개 세로3개 9개사진
#     img_test = plt.imread(all_images_paths[3])
#     plt.imshow(img_test)  # 이미지 보이게 하기
#     plt.title(all_images_paths[c])  # 사진 제목 입력
#     plt.axis('off')  # x좌표 y좌표 삭제
# plt.show()


train_path = []

for image_path in all_images_paths:
    if str(image_path).split('.')[-1] != "mat":
        continue
train_path.append(str(image_path))


    # print("#image_path: {0}".format(image_path))
# print("#train_path", train_path)

# if str(image_path).split('/')[-2] == 'train':
#     train_path.append(str(image_path))
# elif str(image_path).split('/')[-2] == 'val':
#     valid_path.append(str(image_path))
# else:
#     test_path.append(str(image_path))


def get_hr_and_lr(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    hr = tf.image.random_crop(img, [50, 50, 3])
    lr = tf.image.resize(hr, [25, 25])
    lr = tf.image.resize(lr, [50, 50])

    return lr, hr


train_dataset = tf.data.Dataset.list_files(image_path)
train_dataset = train_dataset.map(get_hr_and_lr)
train_dataset = train_dataset.repeat()
train_dataset = train_dataset.batch(16)

valid_dataset = tf.data.Dataset.list_files(image_path)
valid_dataset = valid_dataset.map(get_hr_and_lr)
valid_dataset = valid_dataset.repeat()
valid_dataset = valid_dataset.batch(1)
# 데이터셋 일차 완성

# REDNet-30의 정의


def REDNet(num_layers=15):  # num_layer:컨볼루션 레이어와 디컨볼루션 레이어의 수
    # 같은 수의 컨볼루션 레이어가 존재하기 때문에, REDNet-30이라면 num_layers=15
    conv_layers = []  # 컨볼루션 레이어
    deconv_layers = []  # 디컨볼루션 레이어
    residual_layers = []  # 잔류 레이어

    # 입력 레이어의 shape에서 이미지의 높이와 너비를 None으로 지정해서 어떤 크기의 이미지라도 입력으로 받을 수 있음
    # 첫 번째 컨볼루션 레이어와 마지막 디컨볼루션 레이어를 제외한 레이어들은 for 문 안에서 정의해서 각 리스트에 저장
    # 첫번째 컨볼루션 레이어와 마지막 디컨볼루션 레이어는 필터의 수가 다른데 이는 필터의 수로 RGB 채널의 수인 3을 그대로 받기 위함
    #  나머지 레이어에서는 64개의 필터를 사용

    inputs = tf.keras.layers.Input(shape=(None, None, 3))
    conv_layers.append(tf.keras.layers.Conv2D(
        3, kernel_size=3, padding='same', activation='relu'))

    for i in range(15-1):
        conv_layers.append(tf.keras.layers.Conv2D(
            64, kernel_size=3, padding='same', activation='relu'))
        deconv_layers.append(tf.keras.layers.Conv2DTranspose(
            64, kernel_size=3, padding='same', activation='relu'))

    deconv_layers.append(tf.keras.layers.Conv2DTranspose(
        3, kernel_size=3, padding='same'))

    # 인코더 시작
    x = conv_layers[0](inputs)  # 결과: x는 입력 레이어에 첫 번째 컨볼루션 레이어를 적용한 결과

    for i in range(15-1):
        x = conv_layers[i+1](x)
        if i % 2 == 0:
            residual_layers.append(x)

    # for 문 안에서 x에 나머지 컨볼루션 레이어를 계속 적용시키며, 짝수번재 컨볼루션 레이어를 지날 때마다 x를 잔류 레이어 리스트에도 저장
    # 잔류 레이어에 x를 저장한 다음 스텝에서 x는 다시 컨볼루션 레이어를 통과해서 새로운 값이 되지만 잔류 레이어에 이미 저장된 값은 사라지지 않음

    # 디코더 시작
    for i in range(15-1):
        if i % 2 == 1:
            x = tf.keras.layers.Add()([x, residual_layers.pop()])
            x = tf.keras.layers.Activation('relu')(x)
        x = deconv_layers[i](x)

    x = deconv_layers[-1](x)

    # 홀수 번째의 디컨볼루션 레이어를 통과할 경우 잔류 레이어 리스트에 저장돼 있던 값을 residual_layers.pop()으로 뒤에서부터 하나씩 가져옴
    # 그 다음 합연산과 ReLU 활성화함수를 통과한 후 다음 디컨볼루션 레이어에 연결 (짝수 번째일 때는 디컨볼루션 레이어만 연결)

    # x라는 변수에 레이어를 계속 적용해서 함수형 API를 사용
    # 마지막에 x는 모든 레이어가 적용된 결과가 되기 때문에 모델의 출력 => 하나의 변수 이름을 재사용하여 레이어 적용

    model = tf.keras.Model(inputs=inputs, outputs=x)
    # tf.keras의 함수형 API로 Model을 만들기 위해서는 입력과 출력만 지정하면 됨
    # 입력인 inputs는 함수의 가장 앞에서 정의한 입력 레이어로, 출력인 outputs는 지금까지 레이어 연산을 쭉 따라온 변수 이름인 x로 넣고, model을 반환
    return model


def psnr_metric(y_true, y_pred):
    return tf.image.psnr(y_true, y_pred, max_val=1.0)
# y_true: 정답에 해당하는 값
# y_pred: 네트워크가 학습 결과 예측한 값
# 이 둘의 tf.image.psnr()을 계산해서 반환하는 것이 psnr_metric() 함수의 역할


model = REDNet(15)
model.compile(optimizer=tf.optimizers.legacy.Adam(0.0001),
              loss='mse', metrics=[psnr_metric], run_eagerly=True)
# REDNet() 함수로 네트워크를 초기화하고 컴파일

tf.keras.utils.plot_model(model)
# 컴파일된 네트워크 시각화를 작성

history = model.fit_generator(train_dataset,
                              epochs=1000,
                              steps_per_epoch=len(train_path)//16,
                              validation_data=valid_dataset,
                              validation_steps=len(train_path),
                              verbose=2)
# 네트워크를 학습
# Dataset를 이용한 학습은 fit() 함수 대신 fit_generator() 함수를 사용
# Dataset에 repeat() 함수를 사용했기 때문에 한 번의 에포크에 몇 개의 데이터를 학습시킬지를 지정하는 steps_per_epoch인수를 설정
# batch size가 16이기 때문에 steps_per_epoch는 len(train_path)//16으로 훈련 데이터의 크기를 batch size로 나눕니다.
# verbose = 2 : 출력제한에 걸리지 않도록 하며, 진행 상황 애니메이션은 생략하고 각 에포크의 결과만 출력
# 학습 결과 :  훈련 데이터의 PSNR은 31~32, 검증데이터의 29-32정도

plt.plot(history.history['psnr_metric'], 'b-', label='psnr')
plt.plot(history.history['val_psnr_metric'], 'r--', label='val_psnr')
plt.xlabel('Epoch')
plt.legend()
plt.show()
# 학습 결과 확인

image_path = tf.keras.utils.get_file('butterfly.png', 'http://bit.ly/2oAOxgH')
img = tf.io.read_file(image_path)
img = tf.image.decode_jpeg(img, channels=3)
hr = tf.image.convert_image_dtype(img, tf.float32)

lr = tf.image.resize(hr, [hr.shape[0]//4, hr.shape[1]//4])
lr = tf.image.resize(lr, [hr.shape[0], hr.shape[1]])
predict_hr = model.predict(np.expand_dims(lr, axis=0))

print(tf.image.psnr(np.squeeze(predict_hr, axis=0), hr, max_val=1.0))
print(tf.image.psnr(lr, hr, max_val=1.0))
# PSNR 수치 모두 학습할수록 증가하는 경향 => 이렇게 학습된 데이터가 실제 이미지를 어떻게 복원하는지 확인


plt.figure(figsize=(16, 6))
plt.subplot(1, 3, 1)
plt.imshow(hr)
plt.title('original - hr')

plt.subplot(1, 3, 2)
plt.imshow(lr)
plt.title('lr')

plt.subplot(1, 3, 3)
plt.imshow(np.squeeze(predict_hr, axis=0))
plt.title('sr')

plt.show()
# 테스트 이미지에 대한 초해상도 결과 확인
# 첫번째 사진 : 이미지의 원본, 두번째 사진 : 저해상도, 세번째 사진 : 복원된 이미지


NameError: ignored